<a href="https://colab.research.google.com/github/seo-minsu/machine_learning_study-minsu-/blob/main/Naive_Bayes(Minsu)2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import numpy as np
import pandas as pd

from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.datasets import fetch_covtype, fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn import metrics

In [38]:
# 각 prior의 가능성(확률) 계산

prior = [0.45, 0.3, 0.15, 0.1]
likelihood = [[0.3, 0.3, 0.4], [0.7, 0.2, 0.1], [0.15, 0.5, 0.35], [0.6, 0.2, 0.2]]

idx = 0
for c, xs in zip(prior, likelihood):
  result = 1.

  for x in xs:
    result *= x
  result *= c

  idx +=1

  print(f"{idx}번째 클래스의 가능성: {result}")

1번째 클래스의 가능성: 0.0162
2번째 클래스의 가능성: 0.0042
3번째 클래스의 가능성: 0.0039375
4번째 클래스의 가능성: 0.0024000000000000002


1. 산림 토양 데이터
 * 산림 지역 토양의 특징 데이터
 * 토양이 어떤 종류에 속하는지 예측

In [39]:
covtype = fetch_covtype()
print(covtype.DESCR)

.. _covtype_dataset:

Forest covertypes
-----------------

The samples in this dataset correspond to 30×30m patches of forest in the US,
collected for the task of predicting each patch's cover type,
i.e. the dominant species of tree.
There are seven covertypes, making this a multiclass classification problem.
Each sample has 54 features, described on the
`dataset's homepage <https://archive.ics.uci.edu/ml/datasets/Covertype>`__.
Some of the features are boolean indicators,
while others are discrete or continuous measurements.

**Data Set Characteristics:**

=================   ============
Classes                        7
Samples total             581012
Dimensionality                54
Features                     int
=================   ============

:func:`sklearn.datasets.fetch_covtype` will load the covertype dataset;
it returns a dictionary-like 'Bunch' object
with the feature matrix in the ``data`` member
and the target values in ``target``. If optional argument 'as_frame' is
se

In [40]:
pd.DataFrame(covtype.data)

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2595.0,45.0,2.0,153.0,-1.0,391.0,220.0,234.0,150.0,6172.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396.0,153.0,20.0,85.0,17.0,108.0,240.0,237.0,118.0,837.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581008,2391.0,152.0,19.0,67.0,12.0,95.0,240.0,237.0,119.0,845.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581009,2386.0,159.0,17.0,60.0,7.0,90.0,236.0,241.0,130.0,854.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581010,2384.0,170.0,15.0,60.0,5.0,90.0,230.0,245.0,143.0,864.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
covtype.target

array([5, 5, 2, ..., 3, 3, 3], dtype=int32)

1-1. 학습, 평가 데이터 분류

In [42]:
covtype_X = covtype.data
covtype_y = covtype.target

In [43]:
covtype_X_train, covtype_X_test, covtype_y_train, covtype_y_test = train_test_split(covtype_X, covtype_y, test_size=0.2)

In [49]:
print('전체 데이터 크기: {}'.format(covtype_X.shape))
print('학습 데이터 크기: {}'.format(covtype_X_train.shape))
print('평가 데이터 크기: {}'.format(covtype_X_test.shape))

전체 데이터 크기: (581012, 54)
학습 데이터 크기: (464809, 54)
평가 데이터 크기: (116203, 54)


In [ ]:
1-2. 전처리 전 데이터

In [50]:
covtype_df = pd.DataFrame(data=covtype_X)
covtype_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
count,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,...,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000
mean,2959.365301,155.656807,14.103704,269.428217,46.418855,2350.146611,212.146049,223.318716,142.528263,1980.291226,...,0.044175,0.090392,0.077716,0.002773,0.003255,0.000205,0.000513,0.026803,0.023762,0.015060
std,279.984734,111.913721,7.488242,212.549356,58.295232,1559.254870,26.769889,19.768697,38.274529,1324.195210,...,0.205483,0.286743,0.267725,0.052584,0.056957,0.014310,0.022641,0.161508,0.152307,0.121791
min,1859.000000,0.000000,0.000000,0.000000,-173.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2809.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1024.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2996.000000,127.000000,13.000000,218.000000,30.000000,1997.000000,218.000000,226.000000,143.000000,1710.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3163.000000,260.000000,18.000000,384.000000,69.000000,3328.000000,231.000000,237.000000,168.000000,2550.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3858.000000,360.000000,66.000000,1397.000000,601.000000,7117.000000,254.000000,254.000000,254.000000,7173.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [51]:
covtype_train_df = pd.DataFrame(data=covtype_X_train)
covtype_train_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
count,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,...,464809.000000,464809.000000,464809.000000,464809.00000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000
mean,2959.608986,155.664264,14.100674,269.424787,46.416268,2351.697611,212.174663,223.319662,142.495597,1979.923042,...,0.044474,0.090282,0.077688,0.00279,0.003285,0.000187,0.000521,0.026716,0.023771,0.015028
std,279.846693,111.881094,7.483159,212.396572,58.291687,1560.768744,26.754636,19.759796,38.262150,1323.752985,...,0.206146,0.286586,0.267680,0.05275,0.057223,0.013680,0.022812,0.161253,0.152335,0.121663
min,1859.000000,0.000000,0.000000,0.000000,-173.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2810.000000,58.000000,9.000000,108.000000,7.000000,1104.000000,198.000000,213.000000,119.000000,1024.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2996.000000,127.000000,13.000000,218.000000,30.000000,1998.000000,218.000000,226.000000,143.000000,1710.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3164.000000,261.000000,18.000000,384.000000,69.000000,3331.000000,231.000000,237.000000,168.000000,2550.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3858.000000,360.000000,65.000000,1390.000000,601.000000,7117.000000,254.000000,254.000000,254.000000,7172.000000,...,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [52]:
ovtype_test_df = pd.DataFrame(data=covtype_X_test)
covtype_test_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
count,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,...,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000
mean,-0.001863,0.000199,0.000685,-0.000204,-0.000143,-0.006429,-0.001713,-0.001173,0.000424,-0.002480,...,0.003238,0.004475,-0.000516,-0.001266,0.001666,-0.000599,0.001627,-0.002635,0.000691,-0.004314
std,1.003496,1.000569,1.002835,0.998600,0.999706,0.995955,1.002207,1.003340,0.996861,0.998955,...,1.007170,1.006379,0.999190,0.987992,1.014475,0.978950,1.035521,0.992265,1.002165,0.982732
min,-3.933216,-1.390983,-1.884379,-1.267289,-3.609354,-1.507298,-7.928648,-11.304374,-3.721421,-1.495652,...,-0.214641,-0.314746,-0.290340,-0.052857,-0.056976,-0.014373,-0.022491,-0.166227,-0.155943,-0.124092
25%,-0.537797,-0.872667,-0.681812,-0.759314,-0.676182,-0.801759,-0.529008,-0.522556,-0.614253,-0.724023,...,-0.214641,-0.314746,-0.290340,-0.052857,-0.056976,-0.014373,-0.022491,-0.166227,-0.155943,-0.124092
50%,0.134139,-0.256051,-0.147338,-0.241932,-0.281662,-0.231433,0.218431,0.135489,0.012402,-0.204571,...,-0.214641,-0.314746,-0.290340,-0.052857,-0.056976,-0.014373,-0.022491,-0.166227,-0.155943,-0.124092
75%,0.727443,0.941436,0.520754,0.538845,0.387307,0.616047,0.704266,0.692296,0.665169,0.427380,...,-0.214641,-0.314746,-0.290340,-0.052857,-0.056976,-0.014373,-0.022491,-0.166227,-0.155943,-0.124092
max,3.207886,1.826147,6.934443,5.270539,9.478425,3.034166,1.563820,1.552817,2.910685,3.916319,...,4.658941,3.177161,3.444237,18.918929,17.551312,69.575573,44.462463,6.015868,6.412592,8.058518


1-3. 전처리

In [54]:
scaler = StandardScaler()
covtype_X_train_scale = scaler.fit_transform(covtype_X_train)
covtype_X_test_scale = scaler.transform(covtype_X_test)

1-4.전처리 후 데이터
 * 평균을 0에 가깝게, 표준편차는 1에 가깝게 정규화

In [55]:
covtype_train_df = pd.DataFrame(data=covtype_X_train_scale)
covtype_train_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
count,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,...,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05
mean,7.451688e-16,-5.187565e-17,8.523902e-17,-4.469851e-17,-6.561845e-17,-1.126329e-16,5.338139e-17,1.861164e-17,-1.879355e-16,4.050994e-19,...,-3.142960e-17,-1.031093e-16,5.417631e-17,-1.920018e-17,2.568177e-18,-2.850982e-18,-1.039500e-18,5.579670e-19,-4.202333e-17,-3.968445e-17
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,...,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-3.932904e+00,-1.391338e+00,-1.884323e+00,-1.268500e+00,-3.764113e+00,-1.506758e+00,-7.930397e+00,-1.130173e+01,-3.724196e+00,-1.495691e+00,...,-2.157410e-01,-3.150270e-01,-2.902268e-01,-5.289803e-02,-5.741122e-02,-1.368242e-02,-2.282357e-02,-1.656795e-01,-1.560445e-01,-1.235190e-01
25%,-5.346111e-01,-8.729301e-01,-6.816211e-01,-7.600168e-01,-6.761909e-01,-7.994131e-01,-5.298027e-01,-5.222561e-01,-6.140696e-01,-7.221318e-01,...,-2.157410e-01,-3.150270e-01,-2.902268e-01,-5.289803e-02,-5.741122e-02,-1.368242e-02,-2.282357e-02,-1.656795e-01,-1.560445e-01,-1.235190e-01
50%,1.300392e-01,-2.562032e-01,-1.470869e-01,-2.421171e-01,-2.816231e-01,-2.266178e-01,2.177321e-01,1.356462e-01,1.318283e-02,-2.039076e-01,...,-2.157410e-01,-3.150270e-01,-2.902268e-01,-5.289803e-02,-5.741122e-02,-1.368242e-02,-2.282357e-02,-1.656795e-01,-1.560445e-01,-1.235190e-01
75%,7.303686e-01,9.414982e-01,5.210808e-01,5.394406e-01,3.874267e-01,6.274494e-01,7.036297e-01,6.923327e-01,6.665708e-01,4.306525e-01,...,-2.157410e-01,-3.150270e-01,-2.902268e-01,-5.289803e-02,-5.741122e-02,-1.368242e-02,-2.282357e-02,-1.656795e-01,-1.560445e-01,-1.235190e-01
max,3.210301e+00,1.826367e+00,6.801857e+00,5.275869e+00,9.513952e+00,3.053180e+00,1.563295e+00,1.552666e+00,2.914225e+00,3.922244e+00,...,4.635187e+00,3.174331e+00,3.445581e+00,1.890430e+01,1.741820e+01,7.308647e+01,4.381436e+01,6.035749e+00,6.408430e+00,8.095917e+00


2. 20 NewsGroup
 * 뉴스 기사가 어느 그룹에 속하는지 분류
 * 뉴스 기사는 텍스트 데이터이기 때문에 특별한 전처리 과정이 필요

In [56]:
newsgroup = fetch_20newsgroups()
print(newsgroup.DESCR)

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

=================   ==========
Classes                     20
Samples total            18846
Dimensionality               1